In [95]:
import pandas as pd
from pathlib import Path
import re

import plotly.graph_objs as go
import plotly.express as px

### Solution analysis
This notebooks generates tables and plots for the scenario analysis in the paper.

In [96]:
# Inputs and outputs
results_path = Path("../results")

tables_path = Path("tables")
figures_path = Path("figures")

In [97]:
# Load all displayed cases
cases = [
    "baseline_2019", "today_2024", "future_2030",
    "policy_co2_100EUR", "policy_co2_200EUR", "policy_co2_500EUR", "policy_co2_750EUR","policy_co2_1000EUR", "policy_co2_1500EUR", "policy_co2_2000EUR",
    "policy_thermic_20pct", "policy_thermic_50pct", "policy_thermic_75pct", "policy_thermic_100pct",
    "policy_elez", "policy_electric", "policy_cost_parity" # , "policy_cost_parity_high_range"
]

df_cases = []

for case in cases:
    df_partial = pd.read_parquet(results_path / "scenario_solutions/{}.vehicles.parquet".format(case))
    f_electric = df_partial["vehicle_type"].str.endswith("electric")

    cost = df_partial["energy_cost_EUR"].sum() + df_partial["externalities_cost_EUR"].sum() + df_partial["vehicle_cost_EUR"].sum() + df_partial["driver_cost_EUR"].sum()
    distance = df_partial["distance_m"].sum() * 1e-3
    deliveries = df_partial["deliveries"].sum()
    
    df_cases.append({
        "case": case,

        "energy_EUR": df_partial["energy_cost_EUR"].sum(),
        "externalities_EUR": df_partial["externalities_cost_EUR"].sum(),
        "vehicle_EUR": df_partial["vehicle_cost_EUR"].sum(),
        "driver_EUR": df_partial["driver_cost_EUR"].sum(),

        "cost_per_delivery_EUR": cost / deliveries,
        "cost_per_distance_EUR_km": cost / distance,

        "deliveries": deliveries,
        "distance": distance
    })

df_cases = pd.DataFrame.from_records(df_cases)
df_cases

,case,energy_EUR,externalities_EUR,vehicle_EUR,driver_EUR,cost_per_delivery_EUR,cost_per_distance_EUR_km,deliveries,distance
0,baseline_2019,1154.371963,99.100327,4612.272727,35572.727273,1.535668,2.500598,26984,16571.425618
1,today_2024,2002.550930,129.101685,5674.090909,44313.454545,1.448319,2.634554,35986,19782.929721
2,future_2030,2856.440577,194.316685,8442.727273,61998.181818,1.347803,2.941205,54527,24986.920460
3,policy_co2_100EUR,2826.870655,615.896871,8483.181818,61998.181818,1.355734,2.943697,54527,25112.683944
4,policy_co2_200EUR,2765.198683,1005.121531,8540.909091,62099.818182,1.364664,2.965516,54527,25092.108228
5,policy_co2_500EUR,2631.777369,2038.409519,8471.363636,62404.727273,1.385484,2.943978,54527,25661.293435
6,policy_co2_750EUR,2499.988576,2575.200850,9012.272727,62404.727273,1.402831,2.963876,54527,25808.162588
7,policy_co2_1000EUR,2351.223102,2832.904667,9638.636364,62404.727273,1.416317,2.987173,54527,25853.033107
8,policy_co2_1500EUR,2122.995312,2885.649376,10984.545455,62404.727273,1.437782,3.017600,54527,25980.219113
9,policy_co2_2000EUR,1978.528687,2850.510829,11830.454545,62506.363636,1.451865,3.026902,54527,26154.088488


In [98]:
def replace_name(name):
    if name == "baseline_2019": return "Baseline 2019"
    if name == "today_2024": return "Today 2024"
    if name == "future_2030": return "Future 2030"
    if name == "policy_elez": return "Electric LEZ"
    if name == "policy_electric": return "100\\% Electric"
    if name == "policy_cost_parity": return "Cost parity"

    if name.startswith("policy_co2"):
        return "Carbon tax ({} EUR)".format(name.split("_")[-1].replace("EUR", ""))

    if name.startswith("policy_thermic"):
        return "ICV tax ({}\\%)".format(name.split("_")[-1].replace("pct", ""))

    return name

In [99]:
df_deliveries = df_cases.copy()
df_deliveries["energy_EUR"] /= df_deliveries["deliveries"]
df_deliveries["externalities_EUR"] /= df_deliveries["deliveries"]
df_deliveries["vehicle_EUR"] /= df_deliveries["deliveries"]
df_deliveries["driver_EUR"] /= df_deliveries["deliveries"]

df_deliveries = df_deliveries.melt(
    id_vars = ["case"], 
    value_vars = ["energy_EUR", "externalities_EUR", "vehicle_EUR", "driver_EUR"],
    var_name = "Component", value_name = "Cost [EUR]"
).rename(columns = { "case": "Case" }).assign(N = "Per parcel")

df_distance = df_cases.copy()
df_distance["energy_EUR"] /= df_distance["distance"]
df_distance["externalities_EUR"] /= df_distance["distance"]
df_distance["vehicle_EUR"] /= df_distance["distance"]
df_distance["driver_EUR"] /= df_distance["distance"]

df_distance = df_distance.melt(
    id_vars = ["case"], 
    value_vars = ["energy_EUR", "externalities_EUR", "vehicle_EUR", "driver_EUR"],
    var_name = "Component", value_name = "Cost [EUR]"
).rename(columns = { "case": "Case" }).assign(N = "Per kilometer")

df_plot = pd.concat([df_deliveries, df_distance])

df_plot["Case"] = df_plot["Case"].apply(replace_name)
df_plot["Case"] = df_plot["Case"].str.replace("\\", "")

def update_icv(x):
    return "{} ICV".format(x.split("(")[1].split(")")[0])
f = df_plot["Case"].str.startswith("ICV")
df_plot.loc[f, "Case"] = df_plot.loc[f, "Case"].apply(update_icv)

def update_co2(x):
    return "{} CO2".format(x.split("(")[1].split(")")[0])
f = df_plot["Case"].str.startswith("Carbon")
df_plot.loc[f, "Case"] = df_plot.loc[f, "Case"].apply(update_co2)

df_plot["Component"] = df_plot["Component"].replace({
    "energy_EUR": "Energy", "externalities_EUR": "Externalities",
    "vehicle_EUR": "Vehicles", "driver_EUR": "Driver"
})

figure = px.bar(df_plot, x = "Case", color = "Component", y = "Cost [EUR]", facet_row = "N")


figure.update_yaxes(matches=None)
figure.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))

figure.update_layout(
    width = 600, height = 300,
    margin = dict(l = 0, r = 0, b = 0, t = 0, pad = 0)
)

figure.update_layout(xaxis_title=None)

figure.write_image(figures_path / "cost_components.pdf")
figure